In [1]:
import sqlite3

In [ ]:
  1              2                   3
User           DBMS               Database
Python       Sqlite3   <--SQL--->   FILE
    connection
      cursor            CREATE, INSERT, SELECT

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
cur = con.cursor()

In [ ]:
cur.execute => SQL 1개 실행
cur.executemany => SQL 1개를 N번 반복 실행
cur.executescript => (비표준) N개의 SQL문 실행

In [4]:
cur.execute('''
    CREATE TABLE TEST (
        ID INTEGER,
        NAME CHAR(1)
    );
''')

In [5]:
cur.execute('''
    INSERT INTO TEST (ID, NAME) VALUES (1, 'A');
''')

In [6]:
cur.execute('SELECT * FROM TEST;')

In [7]:
cur.execute('''
    INSERT INTO TEST (ID, NAME) VALUES (2, 'ABC');
''')

In [8]:
cur.execute('SELECT * FROM TEST;')

In [37]:
# cur.fetchone() => 1행
# cur.fetchmany() => N행
# cur.fetchall => 전체

In [9]:
cur.fetchall()

[(1, 'A'), (2, 'ABC')]

In [10]:
cur.execute('INSERT INTO TEST VALUES (3, NULL);')

In [11]:
cur.execute('SELECT * FROM TEST') # =>2
cur.fetchall() # => 1

[(1, 'A'), (2, 'ABC'), (3, None)]

In [12]:
con.close()

In [13]:
con = sqlite3.connect('coffee.db')
cur = con.cursor()

In [19]:
cur.execute('''
    DROP TABLE IF EXISTS CITY
''')

In [20]:
con.execute('''
    CREATE TABLE CITY (
        CNO INTEGER PRIMARY KEY,
        NAME TEXT NOT NULL
    );
''')

In [22]:
con.execute('''
    INSERT INTO CITY (CNO, NAME) VALUES (1, '성북구');
''')

In [23]:
con.execute('''
    INSERT INTO CITY (NAME) VALUES ('강북구');
''')

In [24]:
con.execute('''
    INSERT INTO CITY VALUES (NULL, '동대문구');
''')

In [25]:
# 에러
con.execute('''
    INSERT INTO CITY VALUES (NULL, NULL);
''')
con.execute('''
    INSERT INTO CITY VALUES (3, '노원구');
''')

IntegrityError: NOT NULL constraint failed: CITY.NAME

In [26]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구')]

In [81]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

In [27]:
cur.lastrowid

3

In [91]:
DB Browser for SQLite // 숙제
=> Table 구조 뜨는거 확인하고, 이상한점 찾기 (데이터가 안들어가있음)

In [ ]:
    SQL
DBMS => DB(?)
----    --
Transaction
데이터가 추가/수정(갱신) => 변화가 있을 때
Commit

In [28]:
con.commit()

In [29]:
# Parameter => ?qmark, :named
con.execute('INSERT INTO CITY VALUES (NULL, ?);', ('중구',))

In [30]:
type('중구'), type(('중구')), type(('중구',)), type(['중구'])

(str, str, tuple, list)

In [31]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구'), (4, '중구')]

In [32]:
con.execute('INSERT INTO CITY VALUES (NULL, :name);', {'name':'중랑구'})

In [33]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

[(1, '성북구'), (2, '강북구'), (3, '동대문구'), (4, '중구'), (5, '중랑구')]

In [34]:
cur.executemany('''
INSERT INTO CITY (NAME) VALUES (?)
''', (['노원구'], ['강남구'], ['서초구']))

In [35]:
type((['노원구'], ['강남구'], ['서초구'])), type((['노원구'], ['강남구'], ['서초구'])[0])

(tuple, list)

In [36]:
cur.execute('SELECT * FROM CITY;')
cur.fetchall()

[(1, '성북구'),
 (2, '강북구'),
 (3, '동대문구'),
 (4, '중구'),
 (5, '중랑구'),
 (6, '노원구'),
 (7, '강남구'),
 (8, '서초구')]

In [37]:
cur.executescript('''
    DROP TABLE IF EXISTS SUPPLIER;
    CREATE TABLE SUPPLIER (
        SNO INTEGER PRIMARY KEY,
        NAME VARCHAR(2),
        CNO INTEGER NOT NULL
    );

    DROP TABLE IF EXISTS PART;
    CREATE TABLE PART (
        PNO INTEGER PRIMARY KEY,
        NAME CHAR(1)
    );
''')

In [38]:
p = [{'n':'아메리카노'}, {'n':'카페라테'}, {'n':'에스프레소'}, {'n':'아이스티'}]
#   len(p) => 4, p[첫번째꺼]=> {'n':'아메리카노'}, p[첫번째꺼]['n'] = 아메리카노
cur.executemany('INSERT INTO PART (NAME) VALUES (:n)', p) # 4번 반복, SQL문 1개, 각 반복마다 n키값에 대응되는 value

In [39]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, '아메리카노'), (2, '카페라테'), (3, '에스프레소'), (4, '아이스티')]

In [ ]:
cursor.execute 함수들
1. python 파라미터 2개 넘김(SQL, 변수) # <= 요기
2. dbms SQL 해석
3. db SQL 적용

In [41]:
cur.execute('SELECT CNO FROM CITY WHERE NAME=?', ['성북구'])
cur.fetchall()

[(1,)]

In [42]:
cur.execute('SELECT CNO FROM CITY WHERE NAME LIKE ? ', ['%구'])
cur.fetchall()

[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)]

In [45]:
cur.execute('''
    SELECT * FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO
''', ['%구'])
cur.fetchall()

[(7, '강남구'),
 (2, '강북구'),
 (6, '노원구'),
 (3, '동대문구'),
 (8, '서초구'),
 (1, '성북구'),
 (4, '중구'),
 (5, '중랑구')]

In [49]:
cur.execute('''
    SELECT * FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO
''', ['%구'])
cur.fetchone() # 1개
cur.fetchmany(2) # N개
cur.fetchall() # 전체

[(3, '동대문구'), (8, '서초구'), (1, '성북구'), (4, '중구'), (5, '중랑구')]

In [50]:
# DBMS
# LIMIT 시작위치, 몇개
cur.execute('''
    SELECT * FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO
    LIMIT 0, 1
''', ['%구'])
cur.fetchall()

[(7, '강남구')]

In [54]:
cur.execute('''
    SELECT CNO FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO ASC
    LIMIT 0, 1
''', ['%성북구%'])
# Curosr => DBMS <- DB
cur.fetchone() # record/tuple/row 단위로 가져옴 => Python iterable 객채로 반환
# Cursor 결과 => 읽어들임
cur.fetchone()[0]
# Cursor 결과 X

TypeError: 'NoneType' object is not subscriptable

In [55]:
cur.execute('''
    SELECT CNO FROM CITY WHERE NAME LIKE ?
    ORDER BY NAME, CNO ASC
    LIMIT 0, 1
''', ['%성북구%'])
cno = cur.fetchone()[0]

In [56]:
cur.execute('INSERT INTO SUPPLIER (NAME, CNO) VALUES(?,?)', ('1호점', cno))
cur.execute('INSERT INTO SUPPLIER (NAME, CNO) VALUES(?,?)', ('2호점', cno))
cur.execute('INSERT INTO SUPPLIER (NAME, CNO) VALUES(?,?)', ('3호점', cno))

In [57]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1), (2, '2호점', 1), (3, '3호점', 1)]

In [58]:
cur.execute('''
    INSERT INTO SUPPLIER (NAME, CNO) VALUES (?, (
        SELECT CNO FROM CITY WHERE NAME LIKE ?
        ORDER BY NAME, CNO ASC
        LIMIT 0, 1
    ))
''', ('1호점', '%강북구%'))
    # INSERT INTO SUPPLIER (NAME, CNO) VALUES ('1호점', 2);
    # INSERT INTO SUPPLIER (NAME, CNO) VALUES ('1호점', 2 = (
    #     SELECT CNO FROM CITY WHERE NAME LIKE '%강북구%'
    #     ORDER BY NAME, CNO ASC
    #     LIMIT 0, 1
    # ))

In [59]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1), (2, '2호점', 1), (3, '3호점', 1), (4, '1호점', 2)]

In [60]:
cur.execute('SELECT NAME FROM CITY WHERE CNO > 2')
c = cur.fetchall()

for name in c:
    name = '%'+name[0]+'%'

    # for no in ['1호점','2호점','3호점']:
    #     cur.execute('''
    #         INSERT INTO SUPPLIER (NAME, CNO) VALUES (?, (
    #             SELECT CNO FROM CITY WHERE NAME LIKE ?
    #             ORDER BY NAME, CNO ASC
    #             LIMIT 0, 1
    #         ))
    #     ''', (no, name))
    params = [{'no':'1호점', 'name':name},{'no':'2호점', 'name':name},{'no':'3호점', 'name':name}]
    cur.executemany('''
        INSERT INTO SUPPLIER (NAME, CNO) VALUES (:no, (
            SELECT CNO FROM CITY WHERE NAME LIKE :name
            ORDER BY NAME, CNO ASC
            LIMIT 0, 1
        ))
    ''', params)

In [61]:
cur.execute('SELECT * FROM SUPPLIER')
len(cur.fetchall())

22

In [62]:
cur.execute('SELECT COUNT(PNO) FROM PART')
cur.fetchone()[0]

4

In [66]:
cur.executescript('''
    DROP TABLE IF EXISTS SELLS;
    
    CREATE TABLE SELLS(
        SNO INTEGER NOT NULL,
        PNO INTEGER NOT NULL,
        PRICE INTEGER NOT NULL DEFAULT 0
    )
''')

In [67]:
cur.executemany('INSERT INTO SELLS VALUES(?,?,?)',
[(1,1,10),
(2,1,10),
(3,1,10),
(5,1,10),
(7,1,10),
(9,1,10),
(11,1,10),
(1,2,15),
(4,2,15),
(6,2,15),
(8,2,15),
(10,2,15),
(12,2,15),
(14,2,15),
(1,3,8),
(18,3,8),
(19,3,8),
(22,4,8),
(1,4,20),
(2,4,20),
(3,4,20)])

In [68]:
con.commit()

In [69]:
cur.execute('SELECT NAME FROM CITY')
cur.fetchall()

[('성북구',),
 ('강북구',),
 ('동대문구',),
 ('중구',),
 ('중랑구',),
 ('노원구',),
 ('강남구',),
 ('서초구',)]

In [70]:
cur.execute('INSERT INTO CITY (NAME) VALUES("광진구");')

In [71]:
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME FROM CITY
    INNER JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
''')
len(cur.fetchall()) # 22개
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
''')
# len(cur.fetchall()) # 23개
# cur.fetchall()[-1]

In [72]:
cur.execute('''
    SELECT CITY.NAME, COUNT(SUPPLIER.NAME) FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
    GROUP BY CITY.CNO
    ORDER BY CITY.NAME ASC
''')
cur.fetchall()

[('강남구', 3),
 ('강북구', 1),
 ('광진구', 0),
 ('노원구', 3),
 ('동대문구', 3),
 ('서초구', 3),
 ('성북구', 3),
 ('중구', 3),
 ('중랑구', 3)]

In [73]:
cur.execute('''
    SELECT CITY.NAME, SUPPLIER.NAME, COUNT(SUPPLIER.NAME), SUM(SELLS.PRICE) FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO=SUPPLIER.CNO
    INNER JOIN SELLS
    ON SUPPLIER.SNO=SELLS.SNO
    GROUP BY CITY.NAME, SUPPLIER.NAME
    ORDER BY CITY.CNO ASC
''')
cur.fetchall()

[('성북구', '1호점', 4, 53),
 ('성북구', '2호점', 2, 30),
 ('성북구', '3호점', 2, 30),
 ('강북구', '1호점', 1, 15),
 ('동대문구', '1호점', 1, 10),
 ('동대문구', '2호점', 1, 15),
 ('동대문구', '3호점', 1, 10),
 ('중구', '1호점', 1, 15),
 ('중구', '2호점', 1, 10),
 ('중구', '3호점', 1, 15),
 ('중랑구', '1호점', 1, 10),
 ('중랑구', '2호점', 1, 15),
 ('노원구', '1호점', 1, 15),
 ('강남구', '2호점', 1, 8),
 ('강남구', '3호점', 1, 8),
 ('서초구', '3호점', 1, 8)]

In [ ]:
# SQL중첩 시킴(SQL안에 SQL) => Procedure
# JOIN(INNER=교집합, LEFT/RIGHT=기준테이블, OUTER)
# GROUP BY

In [74]:
for d in con.iterdump():
    print(d)

BEGIN TRANSACTION;
CREATE TABLE CITY (
        CNO INTEGER PRIMARY KEY,
        NAME TEXT NOT NULL
    );
INSERT INTO "CITY" VALUES(1,'성북구');
INSERT INTO "CITY" VALUES(2,'강북구');
INSERT INTO "CITY" VALUES(3,'동대문구');
INSERT INTO "CITY" VALUES(4,'중구');
INSERT INTO "CITY" VALUES(5,'중랑구');
INSERT INTO "CITY" VALUES(6,'노원구');
INSERT INTO "CITY" VALUES(7,'강남구');
INSERT INTO "CITY" VALUES(8,'서초구');
INSERT INTO "CITY" VALUES(9,'광진구');
CREATE TABLE PART (
        PNO INTEGER PRIMARY KEY,
        NAME CHAR(1)
    );
INSERT INTO "PART" VALUES(1,'아메리카노');
INSERT INTO "PART" VALUES(2,'카페라테');
INSERT INTO "PART" VALUES(3,'에스프레소');
INSERT INTO "PART" VALUES(4,'아이스티');
CREATE TABLE SELLS(
        SNO INTEGER NOT NULL,
        PNO INTEGER NOT NULL,
        PRICE INTEGER NOT NULL DEFAULT 0
    );
INSERT INTO "SELLS" VALUES(1,1,10);
INSERT INTO "SELLS" VALUES(2,1,10);
INSERT INTO "SELLS" VALUES(3,1,10);
INSERT INTO "SELLS" VALUES(5,1,10);
INSERT INTO "SELLS" VALUES(7,1,10);
INSERT INTO "SELLS" VALUES(9,1,